<!-- # Διπλωματική Εργασία
## Ταξινόμηση του καρκίνου του μαστού με μεθόδους μηχανικής μάθησης
### Εξαγωγή χαρακτηριστικών με PCA

> Λάζαρος Πανιτσίδης<br />
> Τμήμα Μηχανικών Παραγωγής και Διοίκησης <br />
> Διεθνές Πανεπιστήμιο της Ελλάδος <br />
> lazarospanitsidis@outlook.com -->

# Diploma thesis
## Breast cancer classification using machine learning methods
### Selected Features

> Lazaros Panitsidis<br />
> Department of Industrial Engineering and Management <br />
> International Hellenic University <br />
> lazarospanitsidis@outlook.com

## Contents
1. [Useful Python Libraries](#1)
1. [Data Processing](#2)
1. [Gaussian Naive Bayes](#3)
1. [Linear Discriminant Analysis](#4)
1. [Quadratic Discriminant Analysis](#5)
1. [Ridge Classifier](#6)
1. [Decision Tree Classifier](#7)
1. [Random Forest Classifier](#8)
1. [ADA Boost Classifier (Adaptive Boosting)](#9)
1. [C-Support Vector Classification](#10)
1. [Stochastic Gradient Descent Classifier](#11)
1. [eXtreme Gradient Boosting](#12)
1. [Light Gradient Boosting Machine](#13)
1. [K-Nearest Neighbors Classifier](#14)
1. [Multi-layer Perceptron Classifier](#15)
1. [Summary](#16)

<a id='1'></a>
## 1) Useful Python Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import scipy.stats as stats
import matplotlib.pyplot as plt
import time
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
#import warnings library
import warnings
# ignore all warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

# some of them are not used in this file
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE, RFECV , mutual_info_classif
from sklearn.model_selection import train_test_split, cross_val_score , GridSearchCV , LeaveOneOut,KFold,RandomizedSearchCV,StratifiedKFold, HalvingGridSearchCV
from skopt import BayesSearchCV # https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV , https://scikit-optimize.github.io/stable/auto_examples/bayesian-optimization.html
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score , make_scorer , classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline , Pipeline # https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
from sklearn.preprocessing import StandardScaler , LabelEncoder , MinMaxScaler
from xgboost import XGBClassifier , plot_importance
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier , RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis , QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgbm
from sklearn.neural_network import MLPClassifier
import pygad

<a id='2'></a>
## 2) Data Processing

In [2]:
dataWISC = pd.read_csv('dataWisc.csv')
dataWISC.drop(["id", "Unnamed: 32"], axis = 1, inplace = True)

# Undersampling function
def make_undersample(_df, column):
  dfs_r = {}
  dfs_c = {}
  smaller = 1e1000
  ignore = ""
  for c in _df[column].unique():
    dfs_c[c] = _df[_df[column] == c]
    if dfs_c[c].shape[0] < smaller:
      smaller = dfs_c[c].shape[0]
      ignore = c

  for c in dfs_c:
    if c == ignore:
      continue
    dfs_r[c] = resample(dfs_c[c], 
                        replace=False, # sample without replacement
                        n_samples=smaller,
                        random_state=0)
  return pd.concat([dfs_r[c] for c in dfs_r] + [dfs_c[ignore]])

dataWISC = make_undersample(dataWISC,'diagnosis')

#Description of the dataset

#how many cases are included in the dataset
length = len(dataWISC)
#how many features are in the dataset
features = dataWISC.shape[1]-1 # - diagnosis

# Number of malignant cases
malignant = len(dataWISC[dataWISC['diagnosis']=='M'])

#Number of benign cases
benign = len(dataWISC[dataWISC['diagnosis']=='B'])

#Rate of malignant tumors over all cases
rate = (float(malignant)/(length))*100

print ("There are "+ str(len(dataWISC))+" cases in this dataset")
print ("There are {}".format(features)+" features in this dataset")
print ("There are {}".format(malignant)+" cases diagnosed as malignant tumor")
print ("There are {}".format(benign)+" cases diagnosed as benign tumor")
print ("The percentage of malignant cases is: {:.2f}%".format(rate))

There are 424 cases in this dataset
There are 30 features in this dataset
There are 212 cases diagnosed as malignant tumor
There are 212 cases diagnosed as benign tumor
The percentage of malignant cases is: 50.00%


In [3]:
y = dataWISC.diagnosis                          # M or B 
x = dataWISC.drop('diagnosis',axis = 1 )
target_names=['Benign','Malignant']
le= LabelEncoder()
le.fit(y)
y_le = le.transform(y)

In [5]:
x_new = x[[ 'radius_mean',
            'compactness_mean',
            'concavity_mean',
            'concave points_mean',
            'radius_se',
            'area_worst',
            'concavity_worst',
            'concave points_worst']]
x_new.head(3)

,radius_mean,compactness_mean,concavity_mean,concave points_mean,radius_se,area_worst,concavity_worst,concave points_worst
49,13.49,0.07698,0.04751,0.033840,0.2338,698.8,0.228200,0.128200
285,12.58,0.04216,0.00186,0.002924,0.2719,564.1,0.005579,0.008772
495,14.87,0.08345,0.06824,0.049510,0.2323,783.6,0.170000,0.101700


In [6]:
# https://machinelearningmastery.com/loocv-for-evaluating-machine-learning-algorithms/#:~:text=Given%20the%20improved%20estimate%20of,biased%20estimates%20of%20model%20performance.
# cv = LeaveOneOut()
rng = np.random.RandomState(13) # random number generator , use it in every random state if shuffle=True for different results.Usefull to test a specific algorithm multiple times within a for loop.

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
cv=StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
search_cv = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

originalclass = []
predictedclass = []

def classification_report_with_accuracy_score(y_true, y_pred):
  originalclass.extend(y_true)
  predictedclass.extend(y_pred)
  #print(classification_report(y_true, y_pred, target_names=target_names)) 
  return accuracy_score(y_true, y_pred)

def print_best_params(search):
    print("")
    print("Best hyperparameters : ", search.best_params_)
    print("")
    print("Best estimator : ", search.best_estimator_)
    print("")

In [7]:
# this method didn't work so it will not be used (nested cross validation which returns the best parameters and their scores)

# Following kf is the outer loop
outer_kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=13)
inner_kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=13)
# model = SVC()
# params = {'kernel':['rbf','linear'],'C':[1,10]}
def nested_cv_with_gscv(model,params,x,y):
    outer_loop_accuracy_scores = []
    inner_loop_won_params = []
    inner_loop_accuracy_scores = []

    # Looping through the outer loop, feeding each training set into a GSCV as the inner loop
    for train_index,test_index in outer_kf.split(x,y):
        
        GSCV = GridSearchCV(estimator=model,param_grid=params,cv=inner_kf)
        
        # GSCV is looping through the training data to find the best parameters. This is the inner loop
        GSCV.fit(x[train_index],y[train_index])
        
        # The best hyper parameters from GSCV is now being tested on the unseen outer loop test data.
        pred = GSCV.predict(x[test_index])
        
        # Appending the "winning" hyper parameters and their associated accuracy score
        inner_loop_won_params.append(GSCV.best_params_)
        outer_loop_accuracy_scores.append(accuracy_score(y[test_index],pred))
        inner_loop_accuracy_scores.append(GSCV.best_score_)

    for i in zip(inner_loop_won_params,outer_loop_accuracy_scores,inner_loop_accuracy_scores):
        print (i)

    print('Mean of outer loop accuracy score:',np.mean(outer_loop_accuracy_scores))

# https://github.com/rosscleung/Projects/blob/b9abc20db545d9f483e90a9b046ea50c74f25718/Tutorial%20notebooks/Nested%20Cross%20Validation%20Example.ipynb

* The process by which the best model will be selected is as follows:
1. Evaluation of the default algorithm with 10-fold cross validation
2. Evaluation of the tuned hyperparameter algorithm with nested cross-validation (5-fold Grid Search/Randomized Search inside a 10-fold cross validation)
3. Choosing the best model (from steps 1 and 2) and finding the best hyperparameters

<a id='3'></a>
## 3) [Gaussian Naive Bayes](<https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB>)

* Default hyperparameters

In [8]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_gnb = Pipeline([('scaler', StandardScaler()), ('gnb', GaussianNB())])
score = cross_val_score(clf_gnb, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.908     0.976     0.941       212
   Malignant      0.974     0.901     0.936       212

    accuracy                          0.939       424
   macro avg      0.941     0.939     0.939       424
weighted avg      0.941     0.939     0.939       424

--- Time of execution : 0.06582403182983398 seconds ---


* Νested Cross Validation with Grid Search for hyper-parameter tuning

In [9]:
start = time.time()
param_grid = { 'gnb__var_smoothing': np.logspace(0,-10, num=100) }

search = GridSearchCV(clf_gnb, param_grid=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro')

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.908     0.981     0.943       212
   Malignant      0.979     0.901     0.939       212

    accuracy                          0.941       424
   macro avg      0.944     0.941     0.941       424
weighted avg      0.944     0.941     0.941       424

--- Time of execution : 9.840656995773315 seconds ---


<a id='4'></a>
## 4) [Linear Discriminant Analysis](<https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html>)

* Default hyperparameters

In [10]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_lda = Pipeline([('scaler', StandardScaler()), ('lda', LinearDiscriminantAnalysis())])

score = cross_val_score(clf_lda, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.894     0.953     0.922       212
   Malignant      0.949     0.887     0.917       212

    accuracy                          0.920       424
   macro avg      0.922     0.920     0.920       424
weighted avg      0.922     0.920     0.920       424

--- Time of execution : 0.08078479766845703 seconds ---


* Νested Cross Validation with Grid Search for Model Evaluation

In [11]:
start = time.time()
param_grid = [
    {
        'lda__solver' : ['lsqr','eigen'],
        'lda__shrinkage':[None,'auto']
    },
    {
        'lda__solver' : ['svd'],
        'lda__tol': np.linspace(0, 0.01, num=100)
    }
]

search = RandomizedSearchCV(clf_lda, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro',n_iter=500)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.894     0.953     0.922       212
   Malignant      0.949     0.887     0.917       212

    accuracy                          0.920       424
   macro avg      0.922     0.920     0.920       424
weighted avg      0.922     0.920     0.920       424

--- Time of execution : 7.515499114990234 seconds ---


<a id='5'></a>
## 5) [Quadratic Discriminant Analysis](<https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html>)

* Default hyperparameters

In [12]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_qda = Pipeline([('scaler', StandardScaler()), ('qda', QuadraticDiscriminantAnalysis())])

score = cross_val_score(clf_qda, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.957     0.943     0.950       212
   Malignant      0.944     0.958     0.951       212

    accuracy                          0.950       424
   macro avg      0.951     0.950     0.950       424
weighted avg      0.951     0.950     0.950       424

--- Time of execution : 0.10073089599609375 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [13]:
start = time.time()
param_grid = {
    'qda__reg_param': np.linspace(0, 1, num=100),
    'qda__tol': np.linspace(0, 0.01, num=100)
}

search = RandomizedSearchCV(clf_qda, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro',n_iter=500)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.957     0.943     0.950       212
   Malignant      0.944     0.958     0.951       212

    accuracy                          0.950       424
   macro avg      0.951     0.950     0.950       424
weighted avg      0.951     0.950     0.950       424

--- Time of execution : 39.69619059562683 seconds ---


<a id='6'></a>
## 6) [Ridge Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html#sklearn.linear_model.RidgeClassifier>)

* Default hyperparameters

In [14]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_rc = Pipeline([('scaler', StandardScaler()), ('rg', RidgeClassifier())])

score = cross_val_score(clf_rc, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.898     0.958     0.927       212
   Malignant      0.955     0.892     0.922       212

    accuracy                          0.925       424
   macro avg      0.926     0.925     0.924       424
weighted avg      0.926     0.925     0.924       424

--- Time of execution : 0.13416314125061035 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [15]:
start = time.time()
param_grid = {
    'rg__alpha' : np.linspace(0, 1, num=10),
    'rg__fit_intercept' : [True,False],
    'rg__copy_X' : [True,False],
    'rg__max_iter' : [None],
    'rg__tol' : [0.001],
    'rg__class_weight' : [None,'balanced'],
    'rg__solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
    'rg__positive' : [False]
}

search = RandomizedSearchCV(clf_rc, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro',n_iter=500)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.894     0.958     0.925       212
   Malignant      0.954     0.887     0.919       212

    accuracy                          0.922       424
   macro avg      0.924     0.922     0.922       424
weighted avg      0.924     0.922     0.922       424

--- Time of execution : 43.70612049102783 seconds ---


<a id='7'></a>
## 7) [Decision Tree Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html>)

* Default hyperparameters

In [16]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_tree = Pipeline([('scaler', StandardScaler()), ('tree', DecisionTreeClassifier(random_state=13))])

score = cross_val_score(clf_tree, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.924     0.920     0.922       212
   Malignant      0.920     0.925     0.922       212

    accuracy                          0.922       424
   macro avg      0.922     0.922     0.922       424
weighted avg      0.922     0.922     0.922       424

--- Time of execution : 0.09126710891723633 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [17]:
start = time.time()
param_grid = {
    'tree__criterion' :['gini','entropy'],
    'tree__splitter' : ['best','random'],
    'tree__max_depth': [list(range(2, 20)),None],
    'tree__min_samples_split': list(range(2, 6)),
    'tree__min_samples_leaf': list(range(1, 8)),
    'tree__min_weight_fraction_leaf' : [0.0],
    'tree__max_features': [None, 'sqrt', 'log2'],
    'tree__max_leaf_nodes' : [None],
    'tree__min_impurity_decrease' : [0.0],
    'tree__class_weight' : [None,'balanced'],
    'tree__ccp_alpha' : [0.0],
    'tree__random_state' : [13]
}

search = RandomizedSearchCV(clf_tree, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro',n_iter=1000)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.900     0.929     0.914       212
   Malignant      0.927     0.896     0.911       212

    accuracy                          0.913       424
   macro avg      0.913     0.913     0.913       424
weighted avg      0.913     0.913     0.913       424

--- Time of execution : 51.515055418014526 seconds ---


* Finding the best hyper-parameters

In [18]:
search.fit(x_new, y) # we need this for adaboost

print_best_params(search)
search_results = pd.DataFrame(search.cv_results_)
search_results.sort_values(by='mean_test_score',ascending=False).head(3)


Best hyperparameters :  {'tree__splitter': 'best', 'tree__random_state': 13, 'tree__min_weight_fraction_leaf': 0.0, 'tree__min_samples_split': 2, 'tree__min_samples_leaf': 2, 'tree__min_impurity_decrease': 0.0, 'tree__max_leaf_nodes': None, 'tree__max_features': None, 'tree__max_depth': None, 'tree__criterion': 'gini', 'tree__class_weight': None, 'tree__ccp_alpha': 0.0}

Best estimator :  Pipeline(steps=[('scaler', StandardScaler()),
                ('tree',
                 DecisionTreeClassifier(min_samples_leaf=2, random_state=13))])



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tree__splitter,param_tree__random_state,param_tree__min_weight_fraction_leaf,param_tree__min_samples_split,param_tree__min_samples_leaf,param_tree__min_impurity_decrease,param_tree__max_leaf_nodes,param_tree__max_features,param_tree__max_depth,param_tree__criterion,param_tree__class_weight,param_tree__ccp_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
226,0.012611,0.013762,0.002792,0.001596,best,13,0.0,2,2,0.0,None,None,None,gini,None,0.0,"{'tree__splitter': 'best', 'tree__random_state...",0.929402,0.917601,0.976467,0.929402,0.976190,0.945813,0.025286,1
499,0.004495,0.000452,0.002906,0.000807,best,13,0.0,4,2,0.0,None,None,None,gini,None,0.0,"{'tree__splitter': 'best', 'tree__random_state...",0.929402,0.917601,0.976467,0.929402,0.976190,0.945813,0.025286,1
113,0.005186,0.000747,0.001596,0.000489,best,13,0.0,4,5,0.0,None,None,None,entropy,balanced,0.0,"{'tree__splitter': 'best', 'tree__random_state...",0.917647,0.929324,0.976467,0.929324,0.976177,0.945788,0.025293,3


<a id='8'></a>
## 8) [Random Forest Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html>)

* Default hyperparameters

In [19]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_rf = Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier(random_state=13))])
                       
score = cross_val_score(clf_rf, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.962     0.943     0.952       212
   Malignant      0.944     0.962     0.953       212

    accuracy                          0.953       424
   macro avg      0.953     0.953     0.953       424
weighted avg      0.953     0.953     0.953       424

--- Time of execution : 1.2561113834381104 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [20]:
start = time.time()
param_grid = {
    'rf__bootstrap': [True,False],
    'rf__max_depth': [list(range(5,15)), None],
    'rf__n_estimators' :[100],
    'rf__max_features': [None, 'sqrt', 'log2'],
    'rf__max_leaf_nodes' : [None,list(range(5,15))],
    'rf__min_samples_leaf': list(range(1,10)),
    'rf__min_samples_split': list(range(2, 6)),
    'rf__criterion' :['entropy','gini'],
    'rf__random_state' : [13]
}

search = RandomizedSearchCV(clf_rf, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=1,scoring='f1_macro',n_iter=300)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits


In [21]:
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.948     0.943     0.946       212
   Malignant      0.944     0.948     0.946       212

    accuracy                          0.946       424
   macro avg      0.946     0.946     0.946       424
weighted avg      0.946     0.946     0.946       424

--- Time of execution : 168.3261158466339 seconds ---


<a id='9'></a>
## 9) [ADA Boost Classifier (Adaptive Boosting)](<https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#:~:text=An%20AdaBoost%20%5B1%5D%20classifier%20is,focus%20more%20on%20difficult%20cases.>)

* Default hyperparameters

In [22]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_adaboost = Pipeline([('scaler', StandardScaler()), ('adab', AdaBoostClassifier(random_state=13))])

score = cross_val_score(clf_adaboost, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.951     0.925     0.938       212
   Malignant      0.927     0.953     0.940       212

    accuracy                          0.939       424
   macro avg      0.939     0.939     0.939       424
weighted avg      0.939     0.939     0.939       424

--- Time of execution : 0.8676400184631348 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [23]:
start = time.time()
param_grid = {
    'adab__base_estimator' : [DecisionTreeClassifier(min_samples_leaf=2, min_samples_split=3,random_state=13)],
    'adab__n_estimators' : np.arange(100,210,10),
    'adab__learning_rate' : np.power(10, np.arange(-3, 1, dtype=float)),
    'adab__algorithm' : ['SAMME', 'SAMME.R'],
    'adab__random_state' : [13],
}

search = RandomizedSearchCV(clf_adaboost, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro',n_iter=300)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.943     0.939     0.941       212
   Malignant      0.939     0.943     0.941       212

    accuracy                          0.941       424
   macro avg      0.941     0.941     0.941       424
weighted avg      0.941     0.941     0.941       424

--- Time of execution : 148.87837553024292 seconds ---


<a id='10'></a>
## 10) [C-Support Vector Classification](<https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html>)

* Default hyperparameters

In [24]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_svc = Pipeline([('scaler', StandardScaler()),('svc', SVC())])

score = cross_val_score(clf_svc, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.940     0.958     0.949       212
   Malignant      0.957     0.939     0.948       212

    accuracy                          0.948       424
   macro avg      0.948     0.948     0.948       424
weighted avg      0.948     0.948     0.948       424

--- Time of execution : 0.08186149597167969 seconds ---


* Νested Cross Validation with Grid Search for hyper-parameter tuning

In [25]:
start = time.time()
param_grid = [
    {
        'svc__kernel': ['rbf'], 
        'svc__gamma': [1,1e-1,1e-2, 1e-3, 1e-4,'auto','scale'], 
        'svc__C': [1, 10, 100, 1000],
        'svc__decision_function_shape': ['ovo', 'ovr'],
        'svc__random_state' : [13]
    },
    {
        'svc__kernel': ['linear'], 
        'svc__C': [1, 10, 100, 1000],
        'svc__decision_function_shape': ['ovo', 'ovr'],
        'svc__random_state' : [13]
    },
]

search = GridSearchCV(clf_svc, param_grid=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro')

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.927     0.958     0.942       212
   Malignant      0.956     0.925     0.940       212

    accuracy                          0.941       424
   macro avg      0.942     0.941     0.941       424
weighted avg      0.942     0.941     0.941       424

--- Time of execution : 5.633396625518799 seconds ---


<a id='11'></a>
## 11) [Stochastic Gradient Descent Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html>)

* Default hyperparameters

In [26]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_sgd = Pipeline([('scaler', StandardScaler()), ('sgd', SGDClassifier(random_state=13))])

score = cross_val_score(clf_sgd, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.938     0.920     0.929       212
   Malignant      0.921     0.939     0.930       212

    accuracy                          0.929       424
   macro avg      0.929     0.929     0.929       424
weighted avg      0.929     0.929     0.929       424

--- Time of execution : 0.06582403182983398 seconds ---


* Νested Cross Validation with Grid Search for hyper-parameter tuning

In [27]:
start = time.time()
param_grid = {
    'sgd__average': [True, False],
    'sgd__l1_ratio': np.linspace(0, 1, num=10),
    'sgd__alpha': np.power(10, np.arange(-2, 1, dtype=float)),
    'sgd__random_state' : [13]
}

grid_search = GridSearchCV(clf_sgd, param_grid=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro')

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(grid_search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.948     0.939     0.943       212
   Malignant      0.939     0.948     0.944       212

    accuracy                          0.943       424
   macro avg      0.943     0.943     0.943       424
weighted avg      0.943     0.943     0.943       424

--- Time of execution : 3.517984390258789 seconds ---


<a id='12'></a>
## 12) [eXtreme Gradient Boosting](<https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters>)

* Default hyperparameters

In [28]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_xgb = Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier(random_state=13))])

score = cross_val_score(clf_xgb, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.957     0.953     0.955       212
   Malignant      0.953     0.958     0.955       212

    accuracy                          0.955       424
   macro avg      0.955     0.955     0.955       424
weighted avg      0.955     0.955     0.955       424

--- Time of execution : 0.6044058799743652 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [29]:
# https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook
# https://www.cs.cornell.edu/courses/cs4780/2018sp/lectures/lecturenote19.html
# https://medium.com/data-design/xgboost-hi-im-gamma-what-can-i-do-for-you-and-the-tuning-of-regularization-a42ea17e6ab6

start = time.time()
param_grid = {
        'xgb__booster' : ['gbtree'],
        'xgb__validate_parameters' : [True],
        'xgb__learning_rate' : np.arange(0.01,1.1,0.05),
        'xgb__gamma' : np.arange(0,1.05,0.05),
        'xgb__max_depth' : np.arange(2,11,1),
        'xgb__min_child_weight' : np.arange(1,6,1),
        'xgb__max_delta_step' : np.arange(0,5,1),
        'xgb__subsample' : [0.5],
        'xgb__colsample_bylevel' : [1],
        'xgb__colsample_bynode' : [1],
        'xgb__colsample_bytree' : [1],
        'xgb__reg_lambda' : [0,1],
        'xgb__reg_alpha' : [0],
        'xgb__tree_method' : ['exact'],
        'xgb__scale_pos_weight' : [1],
        'xgb__objective' : ['binary:logistic'], # 'multi:softmax' -> same scores as 'binary:logistic' with grid search
        #'num_class' : [2],
        'xgb__n_estimators' : np.arange(100,210,10),
        'xgb__random_state' : [13]
    }

grid_search = RandomizedSearchCV(clf_xgb, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=1,scoring='f1_macro',n_iter=300)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(grid_search, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits


In [30]:
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.939     0.948     0.944       212
   Malignant      0.948     0.939     0.943       212

    accuracy                          0.943       424
   macro avg      0.943     0.943     0.943       424
weighted avg      0.943     0.943     0.943       424

--- Time of execution : 240.4299190044403 seconds ---


<a id='13'></a>
## 13) [Light Gradient Boosting Machine](<https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html>)

* Default hyperparameters

In [31]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_lgbm = Pipeline([('scaler', StandardScaler()), ('lgbm', lgbm.LGBMClassifier(random_state=13))])

score = cross_val_score(clf_lgbm, x_new, y_le, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.945     0.967     0.956       212
   Malignant      0.966     0.943     0.955       212

    accuracy                          0.955       424
   macro avg      0.955     0.955     0.955       424
weighted avg      0.955     0.955     0.955       424

--- Time of execution : 0.40244412422180176 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

In [32]:
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
# https://neptune.ai/blog/lightgbm-parameters-guide
# https://www.youtube.com/watch?v=5CWwwtEM2TA&ab_channel=PyData & https://github.com/MSusik/newgradientboosting/blob/master/pydata.pdf

start = time.time()
param_grid = {
        'lgbm__boosting_type' : ['gbdt','dart'],
        'lgbm__num_leaves' : np.arange(5,55,1),
        'lgbm__max_depth' : np.arange(2,11,1),
        'lgbm__learning_rate' : np.arange(0.01,1.1,0.05),
        'lgbm__n_estimators' : np.arange(100,210,10),
        'lgbm__objective' : ['binary'],
        'lgbm__min_child_samples' : np.arange(10,35,5),
        'lgbm__subsample' : [0.5],
        'lgbm__reg_lambda' : [0,1],
        'lgbm__reg_alpha' : [0],
        'lgbm__subsample' : [0.5],
        'lgbm__colsample_bytree' : [1],
        'lgbm__scale_pos_weight' : [1],
        'lgbm__random_state' : [13]
    }

grid_search = RandomizedSearchCV(clf_lgbm, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=1,scoring='f1_macro',n_iter=300)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(grid_search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Fitting 5 folds for each of 300 candidates, totalling 1500 fits


In [33]:
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.939     0.948     0.944       212
   Malignant      0.948     0.939     0.943       212

    accuracy                          0.943       424
   macro avg      0.943     0.943     0.943       424
weighted avg      0.943     0.943     0.943       424

--- Time of execution : 89.87398099899292 seconds ---


<a id='14'></a>
## 14) [K-Nearest Neighbors Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html>)

* Default hyperparameters

In [34]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_knn = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

score = cross_val_score(clf_knn, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.922     0.953     0.937       212
   Malignant      0.951     0.920     0.935       212

    accuracy                          0.936       424
   macro avg      0.937     0.936     0.936       424
weighted avg      0.937     0.936     0.936       424

--- Time of execution : 0.09297537803649902 seconds ---


* Νested Cross Validation with Grid Search for hyper-parameter tuning

In [35]:
start = time.time()
param_grid = {
    'knn__n_neighbors': list(range(3,10)),
    'knn__weights': ['uniform','distance'],
    'knn__algorithm' : ['ball_tree', 'kd_tree', 'brute'],
    'knn__leaf_size': [10,20,30,40,50],
    'knn__p': [1,2],
    'knn__metric': ['minkowski','manhattan','chebyshev']
}

grid_search = GridSearchCV(clf_knn, param_grid=param_grid, n_jobs=-1,cv=search_cv,verbose=0,scoring='f1_macro')

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(grid_search, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.931     0.953     0.942       212
   Malignant      0.952     0.929     0.940       212

    accuracy                          0.941       424
   macro avg      0.941     0.941     0.941       424
weighted avg      0.941     0.941     0.941       424

--- Time of execution : 82.81771111488342 seconds ---


<a id='15'></a>
## 15) [Multi-layer Perceptron Classifier](<https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html>)

* Default hyperparameters

In [36]:
start = time.time()
originalclass = []
predictedclass = []
  
# Cross validate
clf_mlp =  Pipeline([('scaler', StandardScaler()),('mlp', MLPClassifier(shuffle=True,random_state=13))])

score = cross_val_score(clf_mlp, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))
print("--- Time of execution : %s seconds ---" % (time.time() - start))

              precision    recall  f1-score   support

      Benign      0.957     0.948     0.953       212
   Malignant      0.949     0.958     0.953       212

    accuracy                          0.953       424
   macro avg      0.953     0.953     0.953       424
weighted avg      0.953     0.953     0.953       424

--- Time of execution : 3.4507648944854736 seconds ---


* Νested Cross Validation with Randomized Search for hyper-parameter tuning

* Tried a wider range of hyperparameters in nested cross validation at first , but over testing, worst attempts were removed (those in comments). Finally, when few hyperparameters remained, they were tested separately with a 10-fold cross-validation.

1. Solver : ADAM

In [37]:
# http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
# https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote20.pdf
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
# https://datascience.stackexchange.com/questions/36049/how-to-adjust-the-hyperparameters-of-mlp-classifier-to-get-more-perfect-performa
# https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html
param_grid = {
    'mlp__hidden_layer_sizes' : [(14,)] , #[(14,),(100,),(14,4),(7,7,)] 
    'mlp__activation' : ['relu'], #['logistic','tanh','relu']
    'mlp__solver' : ['adam'],
    'mlp__alpha' : [1], #10.0 ** -np.arange(1, 7) , np.linspace(0.1,3,20) , *1
    'mlp__batch_size' : [99], # ~297 samples for train at inner cv -> 3 equal size batches , #[16,32,64,128,'auto']
    'mlp__learning_rate' : ['constant'],
    'mlp__learning_rate_init' : [0.1,0.01,0.001],
    'mlp__max_iter' : [1000],
    'mlp__verbose' : [1]
}

search_adam = GridSearchCV(clf_mlp, param_grid=param_grid, n_jobs=-1,cv=search_cv,verbose=1,scoring='f1_macro')

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search_adam, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Iteration 1, loss = 1.05076302
Iteration 2, loss = 1.01209895
Iteration 3, loss = 0.97559823
Iteration 4, loss = 0.93972659
Iteration 5, loss = 0.90595721
Iteration 6, loss = 0.87350846
Iteration 7, loss = 0.84191166
Iteration 8, loss = 0.81245992
Iteration 9, loss = 0.78384408
Iteration 10, loss = 0.75713442
Iteration 11, loss = 0.73171578
Iteration 12, loss = 0.70843870
Iteration 13, loss = 0.68524010
Iteration 14, loss = 0.66363359
Iteration 15, loss = 0.64287727
Iteration 16, loss = 0.62364296
Iteration 17, loss = 0.60494026
Iteration 18, loss = 0.58700077
Iteration 19, loss = 0.56993349
Iteration 20, loss = 0.55385830
Iteration 21, loss = 0.53884568
Iteration 22, loss = 0.52428039
Iteration 23, loss = 0.51127268
Iteration 24, loss = 0.49889330
Iteration 25, loss = 0.48687519
Iteration 26, loss = 0.47587526
Iteration 27, loss = 0.46541722
Iteration 28, loss = 0.45585337
Iteration 29, loss = 0.44656915
Iteration 30, loss = 

In [38]:
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.947     0.929     0.938       212
   Malignant      0.931     0.948     0.939       212

    accuracy                          0.939       424
   macro avg      0.939     0.939     0.939       424
weighted avg      0.939     0.939     0.939       424



2. Solver : SGD

In [39]:
# http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
# https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote20.html
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
# https://datascience.stackexchange.com/questions/36049/how-to-adjust-the-hyperparameters-of-mlp-classifier-to-get-more-perfect-performa
param_grid = {
    'mlp__hidden_layer_sizes' : [(14,)], # [(14,),(100,),(14,4),(7,7,)] 
    'mlp__activation' : ['relu'], #['logistic','tanh','relu'] 
    'mlp__solver' : ['sgd'],
    'mlp__alpha' : [0.1], #10.0 ** -np.arange(1, 7) , np.linspace(0.1,3,20) , 1 ,*0.1
    'mlp__batch_size' : [99], # ~297 samples for train at inner cv -> 3 equal size batches , #[16,32,64,128,'auto']
    'mlp__learning_rate' : ['constant','adaptive'], # [*'constant','invscaling','adaptive']
    'mlp__learning_rate_init' : [0.01],
    'mlp__power_t' : [0.5],
    'mlp__momentum' : [0.8], # np.arange(0.1,1,0.1), *0.8
    'mlp__max_iter' : [1000],
    'mlp__verbose' : [1]
}

search_sgd = RandomizedSearchCV(clf_mlp, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=1,scoring='f1_macro',n_iter=100)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search_sgd, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Iteration 1, loss = 0.96504635
Iteration 2, loss = 0.83082053
Iteration 3, loss = 0.69977523
Iteration 4, loss = 0.59486374
Iteration 5, loss = 0.51999206
Iteration 6, loss = 0.46357253
Iteration 7, loss = 0.42174088
Iteration 8, loss = 0.38875232
Iteration 9, loss = 0.36269612
Iteration 10, loss = 0.34138982
Iteration 11, loss = 0.32357936
Iteration 12, loss = 0.30865194
Iteration 13, loss = 0.29560771
Iteration 14, loss = 0.28453366
Iteration 15, loss = 0.27454853
Iteration 16, loss = 0.26589968
Iteration 17, loss = 0.25846545
Iteration 18, loss = 0.25158269
Iteration 19, loss = 0.24571987
Iteration 20, loss = 0.24016692
Iteration 21, loss = 0.23536054
Iteration 22, loss = 0.23094719
Iteration 23, loss = 0.22693350
Iteration 24, loss = 0.22322746
Iteration 25, loss = 0.21982446
Iteration 26, loss = 0.21670075
Iteration 27, loss = 0.21376624
Iteration 28, loss = 0.21123841
Iteration 29, loss = 0.20864644
Iteration 30, loss = 

In [40]:
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.948     0.953     0.951       212
   Malignant      0.953     0.948     0.950       212

    accuracy                          0.950       424
   macro avg      0.950     0.950     0.950       424
weighted avg      0.950     0.950     0.950       424



In [41]:
originalclass = []
predictedclass = []
  
# Cross validate
clf_mlp_sgd =  Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',MLPClassifier(solver='sgd',alpha=0.1, batch_size=99,
                               hidden_layer_sizes=(14,),learning_rate='constant',
                               learning_rate_init=0.01, max_iter=1000,momentum=0.8,
                               random_state=13, verbose=0))])

score = cross_val_score(clf_mlp_sgd, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.948     0.953     0.951       212
   Malignant      0.953     0.948     0.950       212

    accuracy                          0.950       424
   macro avg      0.950     0.950     0.950       424
weighted avg      0.950     0.950     0.950       424



3. Solver : LBFGS

In [42]:
# http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
# https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote20.html
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
# https://datascience.stackexchange.com/questions/36049/how-to-adjust-the-hyperparameters-of-mlp-classifier-to-get-more-perfect-performa
param_grid = {
    'mlp__hidden_layer_sizes' : [(14,)],  # [(14,),(100,),(14,4),(7,7,)] 
    'mlp__activation' : ['relu'],
    'mlp__solver' : ['lbfgs'],
    'mlp__alpha' : [1], #10.0 ** -np.arange(1, 7) , np.linspace(0.1,3,20) , 1
    'mlp__max_iter' : np.arange(300,500,10) , # np.arange(100,300,10) , [100,200,500,1000]
}

search_lbfgs = RandomizedSearchCV(clf_mlp, param_distributions=param_grid, n_jobs=-1,cv=search_cv,verbose=1,scoring='f1_macro',n_iter=100)

originalclass = []
predictedclass = []
  
# Nested Cross validation
score = cross_val_score(search_lbfgs, x_new, y, scoring=make_scorer(classification_report_with_accuracy_score),cv=cv)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [43]:
print(classification_report(originalclass, predictedclass, target_names=target_names, digits=3))

              precision    recall  f1-score   support

      Benign      0.944     0.953     0.948       212
   Malignant      0.952     0.943     0.948       212

    accuracy                          0.948       424
   macro avg      0.948     0.948     0.948       424
weighted avg      0.948     0.948     0.948       424



<a id='16'></a>
## 16) Summary

* Below are the tables of the specific feature selection method.
* The performance of the algorithms is in descending order.
* All the results are the average values of a 10-fold cross validation.
* The columns contain the accuracy and the average values of precision, recall and f1 score.
* It is observed that the number of samples of Βenign and Μalignant cancer are equal (212 respectively), so the weighted average and the macro average are equal.

<table style="text-align:center">
    <tr>
        <th colspan="6"> Selected Features : Default algorithms</th>
    </tr>
    <tr>
        <th></th>
        <th>precision </th>
        <th>recall</th>
        <th>f1 score</th>
        <th>accuracy</th>
        <th>Execution Time<br>
        (seconds)</th>  
    </tr>
    <tr>
        <th>MLP</th>
        <td>0.976</td>
        <td>0.976</td>
        <td>0.976</td>
        <td>0.976</td>
        <td>3.570</td>
    </tr>
    <tr>
        <th>AdaBoost</th>
        <td>0.974</td>
        <td>0.974</td>
        <td>0.974</td>
        <td>0.974</td>
        <td>0.750</td>
    </tr>
    <tr>
        <th>SVC</th>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.075</td>
    </tr>
    <tr>
        <th>SGD</th>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.064</td>
    </tr>
    <tr>
        <th>Ridge</th>
        <td>0.962</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.073</td>
    </tr>
    <tr>
        <th>LDA</th>
        <td>0.962</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.071</td>
    </tr>
    <tr>
        <th>Random Forest</th>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>1.138</td>
    </tr>
    <tr>
        <th>LGBM</th>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.471</td>
    </tr>
    <tr>
        <th>XGBoost</th>
        <td>0.950</td>
        <td>0.950</td>
        <td>0.950</td>
        <td>0.950</td>
        <td>0.565</td>
    </tr>
    <tr>
        <th>QDA</th>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.063</td>
    </tr>
    <tr>
        <th>KNN</th>
        <td>0.944</td>
        <td>0.943</td>
        <td>0.943</td>
        <td>0.943</td>
        <td>0.081</td>
    </tr>
    <tr>
        <th>GNB</th>
        <td>0.935</td>
        <td>0.934</td>
        <td>0.934</td>
        <td>0.934</td>
        <td>0.052</td>
    </tr>
    <tr>
        <th>Decision Tree</th>
        <td>0.915</td>
        <td>0.915</td>
        <td>0.915</td>
        <td>0.915</td>
        <td>0.058</td>
    </tr>

</table>

<table style="text-align:center">
    <tr>
        <th colspan="6"> Selected Features : Tuned algorithms</th>
    </tr>
    <tr>
        <th></th>
        <th>precision </th>
        <th>recall</th>
        <th>f1 score</th>
        <th>accuracy</th>
        <th>Execution Time<br>
        (seconds)</th>
    </tr>
    <tr>
        <th>MLP</th>
        <td>0.979</td>
        <td>0.979</td>
        <td>0.979</td>
        <td>0.979</td>
        <td>trial and error</td>
    </tr>
    <tr>
        <th>SGD</th>
        <td>0.970</td>
        <td>0.969</td>
        <td>0.969</td>
        <td>0.969</td>
        <td>3.455</td>
    </tr>
    <tr>
        <th>SVC</th>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>6.842</td>
    </tr>
    <tr>
        <th>LGBM</th>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>86.751</td>
    </tr>
    <tr>
        <th>XGBoost</th>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>0.965</td>
        <td>225.429</td>
    </tr>
    <tr>
        <th>LDA</th>
        <td>0.962</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>6.350</td>
    </tr>
    <tr>
        <th>Ridge</th>
        <td>0.962</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>27.177</td>
    </tr>
    <tr>
        <th>Random Forest</th>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>0.960</td>
        <td>153.843</td>
    </tr>
    <tr>
        <th>AdaBoost</th>
        <td>0.958</td>
        <td>0.958</td>
        <td>0.958</td>
        <td>0.958</td>
        <td>162.774</td>
    </tr>
    <tr>
        <th>KNN</th>
        <td>0.956</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>0.955</td>
        <td>80.449</td>
    </tr>
    <tr>
        <th>QDA</th>
        <td>0.947</td>
        <td>0.946</td>
        <td>0.946</td>
        <td>0.946</td>
        <td>24.638</td>
    </tr>
    <tr>
        <th>Decision Tree</th>
        <td>0.934</td>
        <td>0.934</td>
        <td>0.934</td>
        <td>0.934</td>
        <td>42.595</td>
    </tr>
    <tr>
        <th>GNB</th>
        <td>0.930</td>
        <td>0.927</td>
        <td>0.927</td>
        <td>0.927</td>
        <td>6.874</td>
    </tr>

</table>

- As it is seen, some models perform better with default parameters. This can happen for various reasons such as:
    - Unlucky selection of hyperparameters from random search
    - Hyperparameters selected cause overfitting
    - Smaller training sample in the inner loop due to nested cross-validation

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
- Sources for nested cross validation :
    1. [Cross-Validation and Hyperparameter Search in scikit-learn - A Complete Guide](<https://dev.to/balapriya/cross-validation-and-hyperparameter-search-in-scikit-learn-a-complete-guide-5ed8>)
    2. [Nested Cross Validation for Algorithm Selection](<https://vitalflux.com/python-nested-cross-validation-algorithm-selection/>)
    3. [Nested Cross-Validation for Machine Learning with Python](<https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/>)
    4. [Nested cross validation for model selection](<https://stats.stackexchange.com/questions/65128/nested-cross-validation-for-model-selection/65158#65158>)
    5. [scikit-learn GridSearchCV with multiple repetitions](<https://stackoverflow.com/questions/42228735/scikit-learn-gridsearchcv-with-multiple-repetitions/42230764#42230764>)
    6. [Model selection done right: A gentle introduction to nested cross-validation](<https://ploomber.io/blog/nested-cv/>)
    7. [How to obtain optimal hyperparameters after nested cross validation?](<https://stats.stackexchange.com/questions/254612/how-to-obtain-optimal-hyperparameters-after-nested-cross-validation>)
    8. [Cross-validation for parameter tuning, model selection, and feature selection](<https://github.com/justmarkham/scikit-learn-videos/blob/master/07_cross_validation.ipynb>)
- Sources for Hyper Parameter-Optimization :
    1. [Random Search for Hyper-Parameter Optimization](<https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf>)
    2. [Hyperparameter tuning for machine learning models](<https://www.jeremyjordan.me/hyperparameter-tuning/>)
- Sources for code :
    - All sources are in comments at each code part